# Работа с файлами Microsoft Excel и LibreOffice Calc (чтение и запись)

* [1. Использование библиотек xlrd и xlwt](#1.-Использование-библиотек-xlrd-и-xlwt)
  * [1.1. Чтение файла xls, xlsx или ods с помощью библиотеки xlrd](#1.1.-Чтение-файла-xls,-xlsx-или-ods-с-помощью-библиотеки-xlrd)
  * [1.2. Запись в файл xlsx с помощью библиотеки xlwt](#1.2.-Запись-в-файл-xlsx-с-помощью-библиотеки-xlwt)

* [2. Использование библиотеки openpyxl](#2.-Использование-библиотеки-openpyxl)

* [3. Использование COM](#3.-Использование-COM)

* [Работа с файлами и каталогами](#Работа-с-файлами-и-каталогами)

* [Варианты заданий](#Варианты-заданий)

## 1. Использование библиотек xlrd и xlwt

Необходимо установить библиотеку `xlrd` версии 1.2.0 (более поздние версии не поддерживают работу с `xlsx`); для этого в командной строке Anaconda надо выполнить команду:
    
    conda install -c anaconda xlrd=1.2.0 
    
После этого можно будет использовать:

    import xlrd   
    
Документация о пакете `xlrd`: http://xlrd.readthedocs.io/en/latest/api.html

###  1.1. Чтение файла xls, xlsx или ods с помощью библиотеки xlrd

Создадим файл `xlsx` в Microsoft Excel или OpenOffice/LibreOffice Calc. Заполним любой информацией ячеку с адресом B3, а также несколько строк ниже (левые 3-4 колонки).

In [ ]:
# Открываем рабочую книгу Excel (для повышения скорости работы не читаем информацию о формате ячеек):
rb = xlrd.open_workbook("ИмяФайла.xlsx", formatting_info=False) 
print("В рабочай книге имеется {} листов.".format(rb.nsheets))
print("Их имена: {}".format(rb.sheet_names()))
# Начинаем работать с самым первым рабочим листом (они нумеруются с нуля):
sheet = rb.sheet_by_index(0) 
# Другой вариант - не по номеру листа, а по имени: sheet_by_name("ИмяЛиста").

# Читаем содержимое ячейки, указывая номера строки и столбца 
# (нумерация начинается с нуля, поэтому адрес B3 соответствует строке 1 и столбцу 2):
x1 = sheet.cell(1, 2).value.strip() # strip() удаляет лидирующие и конечные пробельные символы в строке, 
#                                     оставляя без изменения внутренние пробелы.
print(x1)

In [ ]:
# Чтение всех строк:
for i in range(sheet.nrows):  # Цикл по номеру строки (начиная с нулевой).
    row = sheet.row_values(i) # Получили список ячеек в i-й строке.
    # Далее можно организовать цикл по всем ячейкам строки или обращаться к ячейке с известным номером столбца.

Чтобы преобразовать строковое представление числа `sx` в само число `x`:

    x = int(sx) if sx else 0
или
    
    x = float(sx) if sx else 0
    
Если строка `sx` пустая, то получается 0. 


Чтобы узнать тип ячейки:

    sheet.cell(1, 2).cell.ctype
    
Возвращаемые значения: `xlrd.XL_CELL_TEXT`, `xlrd.XL_CELL_NUMBER`, `xlrd.XL_CELL_EMPTY` и др. (см. документацию).

###  1.2. Запись в файл xlsx с помощью библиотеки xlwt

Документация: http://xlwt.readthedocs.io/en/latest/.


In [ ]:
import xlwt
from datetime import datetime

style0 = xlwt.easyxf('font: name Times New Roman, color-index red, bold on',
    num_format_str='#,##0.00')
style1 = xlwt.easyxf(num_format_str='D-MMM-YY')

wb = xlwt.Workbook()
ws = wb.add_sheet('MySheet1')
ws.write(0, 0, 1234.56, style0)
ws.write(1, 0, datetime.now(), style1)
ws.write(2, 0, 1)
ws.write(2, 1, 1)
ws.write(2, 2, xlwt.Formula("A3+B3"))
wb.save('ИмяФайла.xls')

## 2. Использование библиотеки openpyxl

Документация по openpyxl: http://openpyxl.readthedocs.io/en/latest/tutorial.html.

In [ ]:
# Открытие рабочей книги, выбор рабочего листа и чтение данных в ячейках:
from openpyxl import load_workbook
wb = load_workbook('ИмяФайла1.xlsx')
print(wb.sheetnames) # Список имён рабочих листов.
ws = wb.worksheets[0]
print(ws["A1"].value)
print(ws.cell(row=1, column=2).value) # Строки и столбцы нумеруются с 1.

In [ ]:
# Создание рабочей книги, запись данных в ячейки и сохранение:
from openpyxl import Workbook
workbook = Workbook()
ws = workbook.active
ws["A1"] = "Hello"
ws["B1"] = "world!"
workbook.save(filename="ИмяФайла2.xlsx")

## 3. Использование COM

In [ ]:
import win32com.client as win32

def openWorkbook(xlapp, xlfile):
    try:        
        xlwb = xlapp.Workbooks(xlfile)            
    except Exception as e:
        try:
            xlwb = xlapp.Workbooks.Open(xlfile)
        except Exception as e:
            print(e)
            xlwb = None                    
    return(xlwb)

filename = "ИмяФайла.xlsx"
try:
    excel = win32.gencache.EnsureDispatch('Excel.Application')
    wb = openWorkbook(excel, filename)
    ws = wb.Worksheets('Лист1') 
    excel.Visible = False
    rg = ws.UsedRange
    print(rg.Cells(1,1))
    rows = rg.Rows # Число строк.
    k = 0
    for row in rows:
        print(k, row)
        k += 1
    print("Прочитано {} строк.".format(k))
    wb.Close(True)
    excel.Quit()        

except Exception as e:
    print(e)

finally:
    ws = None
    wb = None
    excel = None

## Работа с файлами и каталогами

In [ ]:
# Получение списка файлов с заданным расширением, 
# находящихся в заданном каталоге (вместе со всеми его подкаталогами):
import os 
path = "Путь_к_каталогу" 
filelist = []
for root, dirs, files in os.walk(path): 
    for file in files: 
        if(file.endswith(".xlsx") or file.endswith(".xls") or file.endswith(".xlsm")):
            filename = os.path.join(root, file)
            filelist.append(filename)
            print(filename)

## Варианты заданий

---

Задание, общее для всех вариантов:

Релизовать свой вариант задания в виде отдельных функций, выполняющих чтение исходных данных, формирование результата, запись результата в файл,... 

Сначала для чтения и записи файлов Excel использовать библиотеки xlrd и xlwt.
Затем переписать для библиотеки openpyxl. 

Стремиться отделить друг от друга фрагменты кода, зависящие и не зависящие от библиотек Excel.

Не патайтесь сразу решить всю задачу полностью: старайтесь разделить процесс решения задачи на последовательность этапов.

После проверки работоспособности обеих программ реализовать иерархию классов 
для работы с файлами Excel с использованием любого набора библиотек Excel по выбору пользователя (т.е. виртуальный класс Base и три его наследника для каждого варианта используемых библиотек; в классах-наследниках необходимо реализовать методы, которые в базовом классе Base объявлены виртуальными по причине своей зависимости от той библиотеки, которая используется для работы с файлом Excel).

----

Работа должна быть выполнена в группах по 2 человека. Каждый отвечает за свои функции или методы класса.

----


1. Прочитать список сотрудников (файл "Исходные_данные\Сотрудники_(фрагмент).xlsx") и сформировать таблицу по кадровому составу Университета ("Образцы_результатов\Справочники.xlsx", лист "Кафедры").

2. Прочитать список студентов (файл "Исходные_данные\Список-студентов_(фрагмент).xlsx") и сформировать таблицу по кол-ву групп ("Образцы_результатов\Учебная-нагрузка-2022-2023.xlsm", лист "Группы").

3. Читать учебные планы (Исходные_данные\Уч-планы\*.*) и календарный график (Исходные_данные\Календарный-График-2022-2023_(ЗФ).xlsx), сформировать учебные сетки по кафедрам ("Образцы_результатов\Сетки\*.*)  (начать с упрощенного варианта - без учёта календарного графика).

4. (группа 3 чел). Читать уч. планы () и уч.сетки ("Образцы_результатов\Сетки\*.*), построить таблицу соответствия "Имя файла уч.плана" - "Номер кафедры" - "Код дисциплины" - "Номер строки сетки"  (в последней колонке писать -1, если дисциплина пропущена в сетках), а также построить таблицу для лишних дисциплин в сетках: "Номер кафедры" - "Номер строки сетки, которая лишняя".

5. Читая учебные планы, сформировать список дисциплин, преподаваемых в 2022-23 учебном году (см. шаблоны, лист "Дисц22").

6. Читая учебные планы, сформировать список всех преподаваемых дисциплин (см. шаблоны, лист "Дисциплины").

7. Читая учебные планы, сформировать список всех специльностей/направлений и специализаций/профилей  (см. шаблоны, листы "Направления" и "Профили").

8. Читая список студентов, сформировать список групп и подсчитать количество студентов в каждой группе  (см. шаблоны, лист "Группы").

9. Читая план набора на 2022-23 уч. год, сформировать списки будущих групп, разделяя абитуриентов поровну между группами (последняя группа не должна быть самой многочисленной).

ToDo: 10. Читая учебные планы и список переносов дисциплин, сформировать ...

----

1.    Абасов Артур Русланович
2.    Антакова Екатерина Алексеевна
3.    Горпинич Римма Андреевна
4.    Крутилов Илья Павлович
5.    Куракин Федор Павлович
6.    Люткин Сергей Сергеевич
7.    Пономарева Мария Александровна
8.    Серко Сергей Игоревич
9.    Сидоров Владислав Геннадьевич
10.    Смирнова Анастасия Александровна
11.    Соколов Алексей Кириллович
12.    Степанов Владимир Дмитриевич
13.    Худолей Сергей Алексеевич
14.    Шадт Максим Андреевич
15.    Шокин Дмитрий Эдуардович

